In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tfd = tf.contrib.distributions
tfb = tfd.bijectors
from tensorflow.keras import layers


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
tf.enable_eager_execution()

## Settings

In [3]:
DTYPE=tf.float32
NP_DTYPE=np.float32

In [756]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
QMat = np.random.rand(3,5)
# normalize the piecewise PDF such that the integral is 1
QMat = tf.nn.softmax(QMat)
# Insert a 0 to the beginning to match the size to the bin array
QMat = np.insert(QMat,0,0,axis=1)
# Print out values of Q for testing purposes
QMat

array([[0.        , 0.17694308, 0.12909111, 0.22869545, 0.22170349,
        0.24356687],
       [0.        , 0.23328833, 0.25628785, 0.13661589, 0.1210035 ,
        0.25280443],
       [0.        , 0.18656903, 0.13401585, 0.14672721, 0.22498145,
        0.30770645]])

In [777]:
# network in the coupling layer
def net(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return QMat
    return tf.softmax(layers.stack(x, layers.full_connected, [512, 512, out_size]))

class PiecewiseLinear(tfb.Bijector):
    """
    Piecewise Linear: based on 1808.03856
    """
    def __init__(self, D, d, nbins, layer_id=0, validate_args=False, name="PiecewiseLinear"):
        """
        Args:
            D: number of dimensions
            d: First d units are pass-thru units.
        """
        super(PiecewiseLinear, self).__init__(
            forward_min_event_ndims=1, validate_args=validate_args, name=name
        )
        self.D, self.d = D, d
        self.id = layer_id
        self.nbins = nbins
        self.width = 1.0/self.nbins
        
    def Q(self, xd):
        with tf.variable_scope('Q%d' % self.id, reuse=tf.AUTO_REUSE):
            return net(xd, self.D - self.d, self.nbins)
        
    def pdf(self,x):
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.concat([xd, Q[np.arange(len(Q)),ibins]], axis=1)
        
    def _forward(self, x):
        "Calculate forward coupling layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        yD = (xD*self.nbins-ibins)*Q[np.arange(len(Q)),ibins+1]+np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins]
        return tf.concat([xd, yD], axis=1)
        
    def _inverse(self, y):
        "Calculate inverse coupling layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(Q,axis=1),tf.transpose(yD),side='right'))-1
        xD = ((yD-np.cumsum(Q,axis=1)[np.arange(len(Q)),ibins])*tf.reciprocal(Q[np.arange(len(Q)),ibins+1])+np.array(ibins,dtype=np.float32))*self.width
        return tf.concat([yd, xD], axis=1)
    
    def _forward_log_det_jacobian(self, x):
        "Calculate log determinant of Coupling Layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)
    
    def _inverse_log_det_jacobian(self, y):
        "Calculate log determinant of Coupling Layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = np.array(np.floor(yD*self.nbins),dtype=np.int32)
        return -tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)

In [778]:
test = PiecewiseLinear(6,3,5)

In [779]:
forward = test._forward(np.array([[0.1,0.2,0.3,0.4,0.5,0.6],[0.6,0.5,0.4,0.0,0.2,0.1]]).reshape(2,6))
forward

[[2 2 3]
 [0 1 0]]


<tf.Tensor: id=9290, shape=(2, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.30603419, 0.55788413,
        0.4673121 ],
       [0.6       , 0.5       , 0.4       , 0.        , 0.23328833,
        0.09328452]])>

In [780]:
test._inverse(forward)

tf.Tensor(
[[2 2 3]
 [0 1 0]], shape=(2, 3), dtype=int32)


<tf.Tensor: id=9345, shape=(2, 6), dtype=float64, numpy=
array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
       [0.6, 0.5, 0.4, 0. , 0.2, 0.1]])>

In [571]:
inverse = test._inverse(np.array([0.17224058,0.17224058,0.17224058,0.17224058,0.17224058,0.17224058]).reshape(1,6))
inverse

tf.Tensor([[1 1 0]], shape=(1, 3), dtype=int32)


<tf.Tensor: id=7603, shape=(1, 6), dtype=float64, numpy=
array([[ 1.72240580e-01,  1.72240580e-01,  1.72240580e-01,
         2.74970093e-09,  1.68707579e-02, -3.68990786e-03]])>

In [572]:
test._forward(inverse)

<tf.Tensor: id=7626, shape=(1, 6), dtype=float64, numpy=
array([[0.17224058, 0.17224058, 0.17224058, 0.17224058, 0.16482903,
        1.18800735]])>

In [573]:
test.pdf(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=7632, shape=(1, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.1375915 , 0.29325187,
        0.15774332]])>

In [500]:
test._forward_log_det_jacobian(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=6303, shape=(1,), dtype=float64, numpy=array([-0.40662136])>

In [501]:
test._inverse_log_det_jacobian(np.array([[0.1,0.2,0.3,0.4,0.5,0.6]]).reshape(1,6))

<tf.Tensor: id=6310, shape=(1,), dtype=float64, numpy=array([0.40662136])>

In [229]:
WMat = np.random.rand(3,5)
WMat = tf.nn.softmax(WMat)
WMat

<tf.Tensor: id=3191, shape=(3, 5), dtype=float64, numpy=
array([[0.30461836, 0.13723821, 0.19126584, 0.1497618 , 0.21711578],
       [0.21034329, 0.17996762, 0.25946607, 0.21514902, 0.135074  ],
       [0.23739297, 0.16027006, 0.18968912, 0.21996303, 0.19268482]])>

In [230]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
VMat = np.random.rand(3,6)
expV = tf.exp(VMat)
VDenom = np.sum(0.5*(expV[:,0:5]+expV[:,1:6])*WMat,axis=1,keepdims=True)
# normalize the piecewise PDF such that the integral is 1
VMat = np.true_divide(expV,VDenom)
# Print out values of Q for testing purposes
VMat

array([[0.95857261, 0.89965627, 1.01485801, 1.02914925, 1.14464312,
        0.94965425],
       [0.83384493, 1.02240254, 1.2870976 , 0.90594321, 0.7476643 ,
        1.24475353],
       [0.78848073, 0.9873137 , 1.30597736, 0.5866991 , 1.33639998,
        0.88931847]])

In [457]:
# network in the coupling layer
def netV(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return VMat

# network in the coupling layer
def netW(x, outsize, nbins):
    # Note: Need to put in actual matrix, this is just to test
    return WMat

class PiecewiseQuadratic(tfb.Bijector):
    """
    Piecewise Quadratic: based on 1808.03856
    """
    def __init__(self, D, d, nbins, layer_id=0, validate_args=False, name="PiecewiseLinear"):
        """
        Args:
            D: number of dimensions
            d: First d units are pass-thru units.
        """
        super(PiecewiseQuadratic, self).__init__(
            forward_min_event_ndims=1, validate_args=validate_args, name=name
        )
        self.D, self.d = D, d
        self.id = layer_id
        self.nbins = nbins
        
    def W(self, xd):
        with tf.variable_scope('W%d' % self.id, reuse=tf.AUTO_REUSE):
            return netW(xd, self.D - self.d, self.nbins)
    
            
    def V(self, xd):
        with tf.variable_scope('V%d' % self.id, reuse=tf.AUTO_REUSE):
            return netV(xd, self.D - self.d, self.nbins)
           
    def pdf(self,x):
        xd, xD = x[:, :self.d], x[:, self.d:]
        W = self.W(xd)
        V = self.V(xd)
        ibins = tf.transpose(tf.searchsorted(np.cumsum(W,axis=1),tf.transpose(xD),side='right')-1)
        ibinsp1 = ibins+1
        alpha = (xd-np.cumcum(W,axis=1)[np.arange(len(W)),ibins])/W[np.arange(len(W)),ibins]
        result = (V[np.arange(len(V)),ibinsp1]-V[np.arange(len(V)),ibins])*alpha+V[np.arange(len(V)),ibins]
        return tf.concat([xd, result], axis=1)
        
    def _forward(self, x):
        "Calculate forward coupling layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        W = self.W(xd)
        V = self.V(xd)
        WSum = np.cumsum(W,axis=1)
        VSum = np.cumsum(V,axis=1)
        ibins = tf.stack([tf.range(self.d),tf.squeeze(tf.searchsorted(WSum,tf.transpose(xD),side='right'))],axis=1)
        ibinsp1 = tf.stack([tf.range(self.d),tf.squeeze(tf.searchsorted(WSum,tf.transpose(xD),side='right'))+1],axis=1)
        ibinsSum = tf.stack([tf.range(self.d),tf.squeeze(tf.searchsorted(WSum,tf.transpose(xD),side='right'))-1],axis=1)
        alpha = (xD-tf.gather_nd(WSum,ibinsSum))/tf.gather_nd(W,ibins)
        yD = alpha**2/2.0*tf.gather_nd(np.diff(V),ibins)*tf.gather_nd(W,ibins) \
           + alpha*tf.gather_nd(V,ibins)*tf.gather_nd(W,ibins) \
           + (tf.gather_nd(VSum,ibins)+tf.gather_nd(VSum,ibinsSum))*tf.gather_nd(WSum,ibinsSum)/2.0
        return tf.concat([xd, yD], axis=1)
        
    def _inverse(self, y):
        "Calculate inverse coupling layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        W = self.W(yd)
        V = self.V(yd)
        WSum = np.cumsum(W,axis=1)
        VSum = np.cumsum(V,axis=1)
        ibins = tf.transpose(tf.searchsorted(VSum,tf.transpose(yD),side='right'))
        ibinsp1 = ibins+1
        denom = (V[np.arange(len(V)),ibinsp1]-V[np.arange(len(V)),ibins])
        beta = (yD - (VSum[np.arange(len(V)),ibinsp1]+VSum[np.arange(len(V)),ibins])*WSum[np.arange(len(W)),ibins])/(denom*W[np.arange(len(W)),ibins])
        xD = W[np.arange(len(W)),ibins]*(-V[np.arange(len(V)),ibins]/denom+tf.sqrt((V[np.arange(len(V)),ibins]/denom)**2+2*beta))+WSum[np.arange(len(W)),ibins]
        return tf.concat([yd, xD], axis=1)
    
    def _forward_log_det_jacobian(self, x):
        "Calculate log determinant of Coupling Layer"
        xd, xD = x[:, :self.d], x[:, self.d:]
        Q = self.Q(xd)
        ibins = np.array(np.floor(xD*self.nbins),dtype=np.int32)
        return tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)
    
    def _inverse_log_det_jacobian(self, y):
        "Calculate log determinant of Coupling Layer"
        yd, yD = y[:, :self.d], y[:, self.d:]
        Q = self.Q(yd)
        ibins = np.array(np.floor(yD*self.nbins),dtype=np.int32)
        return -tf.reduce_sum(tf.log(Q[np.arange(len(Q)),ibins]/self.width),axis=-1)

In [458]:
test = PiecewiseQuadratic(6,3,5)

In [460]:
forward=test._forward(np.array([0.1,0.2,0.3,0.4,0.5,0.6]).reshape(1,6))
forward

<tf.Tensor: id=6040, shape=(1, 6), dtype=float64, numpy=
array([[0.1       , 0.2       , 0.3       , 0.51865384, 1.10804107,
        1.99007788]])>

In [462]:
test._inverse(forward)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([0, 1, 2])

In [249]:
np.diff(VMat)

array([[-0.05891635,  0.11520175,  0.01429124,  0.11549386, -0.19498887],
       [ 0.18855761,  0.26469506, -0.38115438, -0.15827892,  0.49708923],
       [ 0.19883297,  0.31866366, -0.71927826,  0.74970088, -0.4470815 ]])

In [250]:
VMat

array([[0.95857261, 0.89965627, 1.01485801, 1.02914925, 1.14464312,
        0.94965425],
       [0.83384493, 1.02240254, 1.2870976 , 0.90594321, 0.7476643 ,
        1.24475353],
       [0.78848073, 0.9873137 , 1.30597736, 0.5866991 , 1.33639998,
        0.88931847]])

In [314]:
tf.gather_nd(WMat,[[0,1],[1,3],[2,4]])

<tf.Tensor: id=4101, shape=(3,), dtype=float64, numpy=array([0.13723821, 0.21514902, 0.19268482])>

In [315]:
WMat

<tf.Tensor: id=3191, shape=(3, 5), dtype=float64, numpy=
array([[0.30461836, 0.13723821, 0.19126584, 0.1497618 , 0.21711578],
       [0.21034329, 0.17996762, 0.25946607, 0.21514902, 0.135074  ],
       [0.23739297, 0.16027006, 0.18968912, 0.21996303, 0.19268482]])>